In [1]:
import joblib
import pickle
from tkinter import * 
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
#Loading the Encoded independent variables (x)
df=pd.read_csv('encoded_df.csv')

In [3]:
#Loading the Model
model= joblib.load('score_predictor.pkl')

In [4]:
#Loading the Scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [5]:
df.head()

,runs,wickets,overs,runs_last_5,wickets_last_5,bat_team_Chennai Super Kings,bat_team_Delhi Daredevils,bat_team_Kings XI Punjab,bat_team_Kolkata Knight Riders,bat_team_Mumbai Indians,...,bat_team_Royal Challengers Bangalore,bat_team_Sunrisers Hyderabad,bowl_team_Chennai Super Kings,bowl_team_Delhi Daredevils,bowl_team_Kings XI Punjab,bowl_team_Kolkata Knight Riders,bowl_team_Mumbai Indians,bowl_team_Rajasthan Royals,bowl_team_Royal Challengers Bangalore,bowl_team_Sunrisers Hyderabad
0,1,0,0.1,1,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
1,1,0,0.2,1,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
2,2,0,0.2,2,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,2,0,0.3,2,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
4,2,0,0.4,2,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False


In [6]:
current_teams = ['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Mumbai Indians','Kings XI Punjab',
       'Royal Challengers Bangalore', 'Delhi Daredevils','Sunrisers Hyderabad']

In [7]:
def calculate_last_5(df, current_over):
    df['ball_count'] = (df['overs'].astype(str).str.split('.').apply(lambda x: int(x[0]) * 6 + int(x[1])))
    current_ball_count = (int(str(current_over).split('.')[0]) * 6) + int(str(current_over).split('.')[1])
    start_ball_count = max(current_ball_count - 30, 0)
    last_30_balls = df[(df['ball_count'] > start_ball_count) & (df['ball_count'] <= current_ball_count)]
    
    runs_last_5 = last_30_balls['runs'].sum()
    wickets_last_5 = last_30_balls['wickets'].sum()
    
    return runs_last_5, wickets_last_5

In [8]:
df.columns

Index(['runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5',
       'bat_team_Chennai Super Kings', 'bat_team_Delhi Daredevils',
       'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders',
       'bat_team_Mumbai Indians', 'bat_team_Rajasthan Royals',
       'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
       'bowl_team_Chennai Super Kings', 'bowl_team_Delhi Daredevils',
       'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders',
       'bowl_team_Mumbai Indians', 'bowl_team_Rajasthan Royals',
       'bowl_team_Royal Challengers Bangalore',
       'bowl_team_Sunrisers Hyderabad'],
      dtype='object')

In [9]:
def predict_score():

    global bat_team, bowl_team, runs, wickets, overs
    bat_team= bat_team.get()
    bowl_team= bowl_team.get()
    runs= int(entry3.get())
    wickets= int(entry4.get())
    overs=float(entry5.get())
    runs_last_5, wickets_last_5= calculate_last_5(df, overs)
    input_data = {
        'runs': [runs],
        'wickets': [wickets],
        'overs': [overs],
        'runs_last_5': [runs_last_5],
        'wickets_last_5': [wickets_last_5],
        'bat_team_Chennai Super Kings': 0,
        'bat_team_Delhi Daredevils': 0,
        'bat_team_Kings XI Punjab': 0,
        'bat_team_Kolkata Knight Riders': 0,
        'bat_team_Mumbai Indians': 0,
        'bat_team_Rajasthan Royals': 0,
        'bat_team_Royal Challengers Bangalore': 0,
        'bat_team_Sunrisers Hyderabad': 0,
        'bowl_team_Chennai Super Kings': 0,
        'bowl_team_Delhi Daredevils': 0,
        'bowl_team_Kings XI Punjab': 0,
        'bowl_team_Kolkata Knight Riders': 0,
        'bowl_team_Mumbai Indians': 0,
        'bowl_team_Rajasthan Royals': 0,
        'bowl_team_Royal Challengers Bangalore': 0,
        'bowl_team_Sunrisers Hyderabad': 0
    }
    input_data[f'bat_team_{bat_team}'] = 1
    input_data[f'bowl_team_{bowl_team}'] = 1
    
    input_df = pd.DataFrame(input_data, index=[0])
    input_df = input_df.astype(int)
    
    # Make sure columns are in the same order as during training
    input_df = input_df.reindex(columns=['runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5',
                                         'bat_team_Chennai Super Kings', 'bat_team_Delhi Daredevils',
                                         'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders',
                                         'bat_team_Mumbai Indians', 'bat_team_Rajasthan Royals',
                                         'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
                                         'bowl_team_Chennai Super Kings', 'bowl_team_Delhi Daredevils',
                                         'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders',
                                         'bowl_team_Mumbai Indians', 'bowl_team_Rajasthan Royals',
                                         'bowl_team_Royal Challengers Bangalore', 'bowl_team_Sunrisers Hyderabad'])
    
    # Convert DataFrame to numpy array for prediction
    input_features = input_df.values
    input_features_scaled= scaler.transform(input_features)
    
    # Make prediction
    predicted_score = model.predict(input_features_scaled)
    
    print(f"Predicted Score: {predicted_score[0]}")
    score_display.config(text=f"Prediction: {predicted_score[0]:.0f}")  

In [ ]:
from tkinter import *

root= Tk()
root.geometry('400x400')
root.config(bg='black')
root.title('IPL Score Predictor')

label= Label(root, text='IPL Score Predictor', font=('Times New Roman', 18, 'bold'), bg='black', fg='white')
label.pack(pady=20)

frame1= Frame(root, bg='black')
frame1.pack()

label1= Label(frame1, bg='black', text='Batting Team', font=('Times New Roman', 14, 'bold'))
label1.grid(row=0, column=0)

bat_team= StringVar()
bat_team.set(current_teams[0])

entry1= OptionMenu(frame1, bat_team, *current_teams )
entry1.grid(row=0, column=1, pady=10)

label2= Label(frame1, bg='black', text='Bowling Team', font=('Times New Roman', 14, 'bold'))
label2.grid(row=1, column=0)

bowl_team= StringVar()
bowl_team.set(current_teams[0])

entry2= OptionMenu(frame1, bowl_team, *current_teams )
entry2.grid(row=1, column=1, pady=10)

label3= Label(frame1, text='Runs', bg='black', font=('Times New Roman', 14, 'bold'))
label3.grid(row=2, column=0)

entry3= Entry(frame1, width=15)
entry3.grid(row=2, column=1, pady=10)

label4= Label(frame1, text='Wickets', bg='black',font=('Times New Roman', 14, 'bold'))
label4.grid(row=3, column=0)

entry4= Entry(frame1, width=15)
entry4.grid(row=3, column=1, pady=10)

label5= Label(frame1, text='Overs', bg='black', font=('Times New Roman', 14, 'bold'))
label5.grid(row=4, column=0)

entry5= Entry(frame1, width=15)
entry5.grid(row=4, column=1, pady=10)


button=Button(root, bg='white', text='Predict Score', command=predict_score)
button.pack(pady=10)


score_display = Label(root, text="", font=('Times New Roman', 14, 'bold'), bg='black', fg='white')
score_display.pack()
root.mainloop()